In [ ]:
#code to identify those in the eCare Home cohort who have ever been identified with a diagnosis of anaemia and haemtiinic deficiency for efalls
create or replace table `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_efalls_anaemia_initial_diagnosis` as (
#bring all snomed and ctv3codes for care home cohort in primary into the cte
with a as (
  select
  person_id,
  snomedcode,
  ctv3code,
  dateevent,
  cast(numericvalue as bignumeric) as hb,
  numericunit
  from `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)),
  #join care home entry date from msater table
  a_master as (
    select
    a.*,
    CAST(j.first_episodestartdate AS datetime) first_episodestartdate,
    j.gender_source_value as gender
    from a
    left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` j on
    a.person_id = j.person_id),
#identify all individuals with an acquired anaemia diagnosis snomed codes within 5 years prior to caer home admission without numericvalue 
anaemia_code_acquired_snomed as (
  select
  person_id, 
  snomedcode, 
  dateevent,
  from a_master
  where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Anaemia & haematinic deficiency' and Otherinstructions not like '%Oxford Handbook%' and TimeConstraintyears is not null) AND dateevent < first_episodestartdate AND  datetime_diff(first_episodestartdate,dateevent,year) <=5),

anaemia_code_acquired_ctv as (
  select
  person_id, 
  ctv3code, 
  dateevent,
  from a_master
  where ctv3code in(select CTV3 from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Anaemia & haematinic deficiency' and Otherinstructions not like '%Oxford Handbook%' and TimeConstraintyears is not null) AND dateevent < first_episodestartdate AND  (datetime_diff(first_episodestartdate,dateevent,year) <=5)
),
#identify all individuals with a hereditary anaemia diagnosis snomed codes  prior to caer home admission without numericvalue

anaemia_code_hereditary_snomed as (
  select
  person_id, 
  snomedcode, 
  dateevent,
  from a_master
  where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Anaemia & haematinic deficiency' and Otherinstructions not like '%Oxford Handbook%' and TimeConstraintyears is null) AND dateevent < first_episodestartdate
),

anaemia_code_hereditary_ctv as (
  select
  person_id, 
  ctv3code, 
  dateevent,
  from a_master
  where ctv3code in(select CTV3 from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Anaemia & haematinic deficiency' and Otherinstructions not like '%Oxford Handbook%' and TimeConstraintyears is null) AND dateevent < first_episodestartdate
),
#identify all males with a measured hb within 5 years whre hb is below the reference value for males
anaemia_male as
(
  select
  person_id,
  snomedcode,
  dateevent,
  from a_master
  where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Anaemia & haematinic deficiency' and Otherinstructions like '%Oxford Handbook%') AND dateevent < first_episodestartdate and gender = 'M' and (hb between 25.0 and 130 or hb <13.0) AND  datetime_diff(first_episodestartdate,dateevent,year) <=5),

anaemia_female as
(
  select
  person_id,
  snomedcode,
  dateevent,
  from a_master
  where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Anaemia & haematinic deficiency' and Otherinstructions like '%Oxford Handbook%') AND dateevent < first_episodestartdate and  gender = 'F' and (hb between 25.0 and 115 OR hb < 11.5) and  datetime_diff(first_episodestartdate,dateevent,year) <=5 ),
# union all different classification of anaemia diagnosis together with a union all and then sequence them to find the initial diagnosis

anaemia_union as (
  select
  * from anaemia_male

  union all

  select
  * from anaemia_code_acquired_ctv

  union all
  
  select 
  * from anaemia_code_acquired_snomed

  union all

  select
  * from anaemia_code_hereditary_snomed

union all

  select
  * from anaemia_code_hereditary_ctv

  union all

  select 
  * from anaemia_female
),

#sequences them to find the initila code within 5 years 
anaemia_union_sequence as (
select
*,
row_number() over (partition by person_id order by dateevent) as code_seq
from anaemia_union)
#identifies all individuals in the CB cohort who have ever fulfilled the anaemia diagnosis

select
person_id, snomedcode as initial_diagnosis_code,dateevent as diagnosis_date
from anaemia_union_sequence
where code_seq = 1 and snomedcode is not null
order by person_id)


